In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
sub = pd.read_csv("../input/sample_submission.csv")
structure = pd.read_csv("../input/structures.csv")

In [ ]:
print(train.shape)
train.head(3)

In [ ]:
print(len(train["type"].unique()))
train["type"].unique()

In [ ]:
train.describe()

In [ ]:
test.head(3)

In [ ]:
print(structure.shape)
structure.head(3)

In [ ]:
print(len(structure["atom"].unique()))
structure["atom"].unique()

In [ ]:
import seaborn as sns

sns.countplot(train["atom_index_0"]);

In [ ]:
sns.countplot(train["atom_index_1"]);

In [ ]:
sns.countplot(train["type"]);

In [ ]:
sns.distplot(train["scalar_coupling_constant"], bins=20);

In [ ]:
sns.violinplot(x="type", y="scalar_coupling_constant", data=train);

In [ ]:
train.head()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (20, 12))
for idx, val in enumerate(train["type"].unique()):
    train_type = train.loc[train["type"] == val]
    G = nx.from_pandas_edgelist(
        df=train_type, 
        source="atom_index_0", 
        target="atom_index_1", 
        edge_attr=["scalar_coupling_constant"]
    )
    
    plt.subplot(2, 4, idx+1);
    nx.draw(G, with_labels=True);
    plt.title(f"Graph for type {t}")

In [ ]:
fig, ax = plt.subplots(figsize = (20, 12))
for idx, val in enumerate(train["type"].unique()):
    train_type = train.loc[train["type"] == val]
    
    rare_atoms_0 = list(
        train_type["atom_index_0"]
        .value_counts(normalize=True)
        [train_type["atom_index_0"]
         .value_counts(normalize=True) < 0.01]
        .index
    )
    
    rare_atoms_1 = list(
        train_type["atom_index_1"]
        .value_counts(normalize=True)
        [train_type["atom_index_1"]
        .value_counts(normalize=True) < 0.01]
        .index
    )
    
    rare_atoms = list(set(rare_atoms_0 + rare_atoms_1))
    
    train_type = train_type.loc[
        (train_type["atom_index_0"].isin(rare_atoms_0) == False) &
        (train_type["atom_index_1"].isin(rare_atoms_1) == False)
    ]
    
    G = nx.from_pandas_edgelist(
        df=train_type, 
        source="atom_index_0", 
        target="atom_index_1", 
        edge_attr=["scalar_coupling_constant"]
    )
    
    plt.subplot(2, 4, idx+1);
    nx.draw(G, with_labels=True);
    plt.title(f"Graph for type {t}")

In [ ]:
join_atom0_to_train = pd.merge(
    train, 
    structure, 
    left_on=["molecule_name", "atom_index_0"],
    right_on=["molecule_name", "atom_index"],
    how="left",
)

join_atom0_to_train.head(3)

In [ ]:
join_atom0_to_train.isnull().sum()

In [ ]:
join_atom0_to_train = join_atom0_to_train.drop("atom_index", axis=1)
join_atom0_to_train = join_atom0_to_train.rename(
    columns={
        "atom": "atom_0",
        "x": "x_0",
        "y": "y_0",
        "z": "z_0",
    }
)

join_atom0_to_train.head(3)

In [ ]:
join_allatom_to_train = pd.merge(
    join_atom0_to_train, 
    structure, 
    left_on=["molecule_name", "atom_index_1"],
    right_on=["molecule_name", "atom_index"],
    how="left",
)

join_allatom_to_train.head(3)

In [ ]:
join_allatom_to_train.isnull().sum()

In [ ]:
join_allatom_to_train = join_allatom_to_train.drop("atom_index", axis=1)
join_allatom_to_train = join_allatom_to_train.rename(
    columns={
        "atom": "atom_1",
        "x": "x_1",
        "y": "y_1",
        "z": "z_1",
    }
)

join_allatom_to_train.head(3)

In [ ]:
train_all = join_allatom_to_train

In [ ]:
join_atom0_to_test = pd.merge(
    test, 
    structure, 
    left_on=["molecule_name", "atom_index_0"],
    right_on=["molecule_name", "atom_index"],
    how="left",
)

join_atom0_to_test.head(3)

In [ ]:
join_atom0_to_test = join_atom0_to_test.drop("atom_index", axis=1)
join_atom0_to_test = join_atom0_to_test.rename(
    columns={
        "atom": "atom_0",
        "x": "x_0",
        "y": "y_0",
        "z": "z_0",
    }
)

join_atom0_to_test.head(3)

In [ ]:
join_allatom_to_test = pd.merge(
    join_atom0_to_test, 
    structure, 
    left_on=["molecule_name", "atom_index_1"],
    right_on=["molecule_name", "atom_index"],
    how="left",
)

join_allatom_to_test.head(3)

In [ ]:
join_allatom_to_test = join_allatom_to_test.drop("atom_index", axis=1)
join_allatom_to_test = join_allatom_to_test.rename(
    columns={
        "atom": "atom_1",
        "x": "x_1",
        "y": "y_1",
        "z": "z_1",
    }
)

join_allatom_to_test.head(3)